# Tutorial IV: Convolutions

<p>
Bern Winter School on Machine Learning, 2-6 March 2020<br>
Prepared by Mykhailo Vladymyrov.
</p>

This work is licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

In this session we will look at the convolutoin operation and try to build some intuition about it.
Also we will look at one of the state-of-the art deep models, [Inception](https://arxiv.org/abs/1602.07261). It is designed to perform image recognition.

## 1. Load necessary libraries

In [ ]:
colab = True # set to True is using google colab

In [ ]:
if colab:
    %tensorflow_version 2.x

In [ ]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipyd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from PIL import Image

# We'll tell matplotlib to inline any drawn figures like so:
%matplotlib inline
plt.style.use('ggplot')


from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 5px;
    color: #0000aa;
    background-color: #cccccc;
} </style>""")
%load_ext tensorboard

### Download libraries

In [ ]:
if colab:
    p = tf.keras.utils.get_file('./material.tgz', 'https://scits-training.unibe.ch/data/tut_files/tpub0320.tgz')
    !mv {p} .
    !tar -xvzf material.tgz > /dev/null 2>&1

In [ ]:
from utils import gr_disp
from utils import inception

In [ ]:
def show_graph(g=None, gd=None):
    gr_disp.show_graph(g, gd)
    %tensorboard --logdir logs

## 2. Convolutions

In fully connected network all inputs of a layer are connected to all neurons of the following layer:
<tr>
    <td> <img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/Perceptron.png" alt="drawing" width="30%"/></td> 
    <td> <img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/MLP.png" alt="drawing" width="50%"/></td> 
</tr> 
<br>In convolutional nets the same holds for each neighbourhood, and the weights are shared:<br>
<img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/CNN1.png" alt="drawing" width="50%"/><br>
<img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/CNN2.png" alt="drawing" width="50%"/><br>
<img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/CNN3.png" alt="drawing" width="50%"/><br>


Let's see what a convolution is, and how it behaves.

In [ ]:
#load image, convert to gray-scale and normalize
img_raw = plt.imread('ML3/chelsea.jpg').mean(axis=2)[-256:, 100:356].astype(np.float32)
img_raw = (img_raw-img_raw.mean())/img_raw.std()

plt.imshow(img_raw, cmap='gray')
plt.grid(False)
img_raw4d = img_raw[np.newaxis,...,np.newaxis]

In [ ]:
g = tf.Graph()
with g.as_default():
    dilation = 1
    stride = 1
    #convolve x 4 times with a 5x5 filter
    x = tf.placeholder(dtype=tf.float32, shape=(1,256,256,1),name='img')
    flt = tf.placeholder(dtype=tf.float32, shape=(None,None,1,1), name='flt')
    
    y1 = tf.nn.conv2d(x , flt, strides=[1,stride,stride,1], dilations=[1,dilation,dilation,1], padding='VALID', name='convolved')
    y2 = tf.nn.conv2d(y1, flt, strides=[1,stride,stride,1], dilations=[1,dilation,dilation,1], padding='VALID', name='convolved')
    y3 = tf.nn.conv2d(y2, flt, strides=[1,stride,stride,1], dilations=[1,dilation,dilation,1], padding='VALID', name='convolved')
    y4 = tf.nn.conv2d(y3, flt, strides=[1,stride,stride,1], dilations=[1,dilation,dilation,1], padding='VALID', name='convolved')
    

In [ ]:
flt_mtx = [
    [ 0, 0, 0, 0, 0,],
    [ 0, 0, 0, 0, 0,],
    [ 0, 0, 1, 0, 0,],
    [ 0, 0, 0, 0, 0,],
    [ 0, 0, 0, 0, 0,],
] # identity transformation

#flt_mtx = gf*gf.transpose()    # gaussian filter
#flt_mtx = dgf*gf.transpose()   # gabor filter, 1st derivative of gaussian (edge detection)
#flt_mtx = ddgf*gf.transpose()  # 2nd derivative of gaussian (line detection)
#flt_mtx = ddgf2d               # 2nd derivative of gaussian, central symmetric, (spot detection)

with tf.Session(graph=g) as sess:
    flt_mtx_np = np.array(flt_mtx, np.float32)
    flt_mtx_np = flt_mtx_np[..., np.newaxis, np.newaxis]
    res = sess.run([x,y1,y2,y3,y4], feed_dict={x:img_raw4d, flt:flt_mtx_np})
res = [r[0,...,0] for r in res]


n = len(res)
fig, ax = plt.subplots(1, n+1, figsize=(n*4, 4))
for col in range(n):
    ax[col].imshow(res[col], cmap='gray')
    ax[col].grid(False)
    ax[col].set_title('conv %d'% col if col else 'raw')

ax[n].imshow(flt_mtx, cmap='gray')
ax[n].grid(False)
_=ax[n].set_title('filter')

In [ ]:
def gaussian(n=5):
    x = np.linspace(-3, 3, n)
    y = np.exp(-x**2 * 0.5) / np.sqrt(2*np.pi)
    return y

def dgaussian(n=5):
    x = np.linspace(-3, 3, n)
    y = - 2 * x * np.exp(-x**2 * 0.5) / np.sqrt(2*np.pi)
    return y

def ddgaussian(n=5):
    x = np.linspace(-3, 3, n)
    y = - 2 * (2*x**2 - 1) * np.exp(-x**2 * 0.5) / np.sqrt(2*np.pi)
    return y
  
def ddgaussian2d(n=5):
    c = np.linspace(-3, 3, n)
    r = np.asarray([[np.sqrt(xi**2+yi**2) for xi in c] for yi in c])
    f = lambda x: (- 2 * (2*x**2 - 1) * np.exp(-x**2 * 0.5) / np.sqrt(2*np.pi))

    y = f(r)
    y -= y.mean()
    return y
  

In [ ]:
n = 10
gf = np.tile(gaussian(n)[np.newaxis], [n, 1])

dgf = np.tile(dgaussian(n)[np.newaxis], [n, 1])

ddgf = ddgaussian(n)
ddgf -= ddgf.mean()
ddgf = np.tile(ddgf[np.newaxis], [n, 1])

ddgf2d = ddgaussian2d(n)
rf2d = lambda:  np.random.normal(size=(5,5))


plt.plot(gf[0])
plt.plot(dgf[0])
plt.plot(ddgf[0])


In [ ]:
plt.imshow(dgf*gf.transpose())
plt.grid(False)

## 3. Homework

In last session we used fully connected network to clasify digits.
Try to build the convolutional network: use three convolutional layers, then flatten the ouput and apply 1 fully connected.
You can use the following helper function. Notice: there is a stride parameter. It allows to effectively downscale the feature maps.
To get an understanding of different convolution types, check the <a href="https://github.com/vdumoulin/conv_arithmetic">animations here</a>.

In [ ]:
def conv_2D(x, n_output_ch,
            k=3,
            s=1,
            activation=tf.nn.relu,
            padding='VALID', name='conv2d', reuse=None
           ):
    """
    Helper for creating a 2d convolution operation.

    Args:
        x (tf.Tensor): Input tensor to convolve.
        n_output_ch (int): Number of filters.
        k (int): Kernel width and height
        s (int): Stride in x and y
        activation (tf.Function): activation function to apply to the convolved data
        padding (str): Padding type: 'SAME' or 'VALID'
        name (str): Variable scope
        reuse (tf.Flag): Flag whether to use existing variable. Can be False(None), True, or tf.AUTO_REUSE

    Returns:
        op (tf.Tensor, tf.Tensor, tf.Tensor, tf.Tensor): Output of activation, convolution, weights, bias
    """
    with tf.variable_scope(name or 'conv2d', reuse=reuse):
        w = tf.get_variable(name='W',
                            shape=[k, k, x.get_shape()[-1], n_output_ch],
                            initializer=tf.initializers.he_uniform()
                           )
        
        wx = tf.nn.conv2d(name='conv',
                          input=x, filter=w,
                          strides=[1, s, s, 1],
                          padding=padding
                         )
        
        b = tf.get_variable(name='b',
                            shape=[n_output_ch], initializer=tf.initializers.constant(value=0.0)
                           )
        h = tf.nn.bias_add(name='h',
                           value=wx,
                           bias=b
                          )

        if activation is not None:
            x = activation(h, name=activation.__name__)
        else:
            x = h
    
    return x, w

You can start with something like this:


In [ ]:
...
x_train = x_train_2d[..., np.newaxis]  # we need additional channel dimension

....

X = tf.placeholder(name='X', dtype=tf.float32, shape=[None, n_input])

L1, W1 = conv_2D(X, 16, name = 'C1')
L2, W2 = conv_2D(L1, 32, s=2, name = 'C2')
L3, W3 = conv_2D(L2, 32, s=2, name = 'C3')

L3_f = tf.keras.layers.Flatten(L3)

L4, W4 = fully_connected_layer(L3_f , 32, 'F1', activation=tf.nn.relu)
L5, W5 = fully_connected_layer(L4 , 10, 'F2')

Y_onehot = tf.nn.softmax(L5, name='Prob')

Play with layer parameters. Can you get better performance than in fully connected network?

## 4. Load the model

inception module here is a small module that performs loading the inception model as well as image preparation for the training.

In [ ]:
net, net_labels = inception.get_inception_model()

In [ ]:
#get model graph definition and change it to use GPU
gd = net

str_dg = gd.SerializeToString()
#uncomment next line to use GPU acceleration
#str_dg = str_dg.replace(b'/cpu:0', b'/gpu:0') #a bit extreme approach, but works =)
gd = gd.FromString(str_dg)

In [ ]:
show_graph(gd=gd) # pass graph def instead of graph itself

## 5. Create the graph

This whole model won't fit in GPU memory. We will take only the part from input to the main output and copy it to a second graph, that we will use further.

In [ ]:
gd2 = tf.graph_util.extract_sub_graph(gd, ['output'])
g2 = tf.Graph() # full graph
with g2.as_default():
    tf.import_graph_def(gd2, name='inception')

show_graph(g2)

## 6. Test the model

We will use one image to check model. `img_preproc` is croped to 256x256 pixels and slightly transformed to be used as imput for the model using `inception.prepare_training_img`. `inception.training_img_to_display` is then used to convert it to displayable one.


In [ ]:
img_raw = plt.imread('ML3/chelsea.jpg')
img_preproc = inception.prepare_training_img(img_raw)
img_deproc = inception.training_img_to_display(img_preproc)
_, axs = plt.subplots(1, 2, figsize=(10,5))
axs[0].imshow(img_raw)
axs[0].grid(False)
axs[1].imshow(img_deproc)
axs[1].grid(False)
plt.show()

We then get the input and output tensors, and obtain probabilities of each class on this image:

In [ ]:
# From graph we will get the input and output tensors. 
# Any tensor and operation can be obtained by name
g2.device('/gpu:0')
with g2.as_default():
    x = g2.get_tensor_by_name('inception/input:0')
    softmax = g2.get_tensor_by_name('inception/output:0')
    
# Then we will feed the image in the graph and print 5 classes that have highest probability
with tf.Session(graph=g2) as sess:
    res = np.squeeze(sess.run(softmax, feed_dict={x: img_preproc[np.newaxis]}))
    
    indexes_sorted_by_probability = res.argsort()[::-1]
    print([(res[idx], net_labels[idx])
           for idx in indexes_sorted_by_probability[:5]])